In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.stem import RSLPStemmer

In [2]:
def orengo(song):
    st = RSLPStemmer()
    words = filter(lambda x: len(x)>0, song)
    word_list = list(map(lambda x: st.stem(x), words))
    return word_list

In [3]:
def remove_stopwords(doc):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    new_doc = list(filter(lambda x: x not in stopwords, doc))
    return new_doc

In [4]:
df = pd.read_csv('.\data\data.csv')

In [5]:
df.head()

,artist,music_name,lyrics,translated,foreign
0,/marilia-mendonca/,/marilia-mendonca/ausencia.html,sei bem o que te faz bem eu sei mas no fundo e...,sei bem o que te faz bem eu sei mas no fundo e...,2
1,/marilia-mendonca/,/marilia-mendonca/todo-mundo-vai-sofrer.html,a garrafa precisa do copo o copo precisa da me...,a garrafa precisa do copo o copo precisa da me...,5
2,/marilia-mendonca/,/marilia-mendonca/bebi-liguei.html,acordei mais uma vez embriagado e o seu cheiro...,acordei mais uma vez embriagado e o seu cheiro...,22
3,/marilia-mendonca/,/marilia-mendonca/ciumeira.html,no começo eu entendia mas era só cama não tinh...,no começo eu entendia era só cama não tinha am...,11
4,/marilia-mendonca/,/marilia-mendonca/bem-pior-que-eu.html,mente que vai dar uma volta e vem me ver entre...,mente que vai dar uma volta e vem me ver entre...,3


In [6]:
df = df.fillna('')

In [7]:
#creates new df where the column lyrics carries every song of a given artist
df1 = df.groupby('artist').agg({'foreign': 'sum', 'lyrics': lambda x: ' '.join(x), 'translated': lambda x: ' '.join(x)}).reset_index()

In [8]:
df1.head()

,artist,foreign,lyrics,translated
0,/aline-barros/,443,mestre eu preciso de um milagre transforma min...,mestre eu preciso de um milagre transforma min...
1,/amado-batista/,297,ao te ver pela primeira vez eu tremi todo uma ...,ao te ver pela primeira vez eu tremi todo uma ...
2,/ana-carolina/,535,eu e você não é assim tão complicado não é dif...,eu e você não é assim tão complicado não é dif...
3,/anavitoria/,681,eu poderia acordar sem teu olhar de sono sem t...,eu poderia acordar sem teu olhar de sono sem t...
4,/anderson-freire/,505,não consigo ir além do teu olhar tudo que eu c...,não consigo ir além do teu olhar tudo que eu c...


In [9]:
#removes stopwords and creates new column
df1['no_stopwords'] = df1['lyrics'].apply(lambda x: remove_stopwords(x.split(' ')))

In [10]:
#applies stemmer and creates new column
df1['orengo'] = df1['lyrics'].apply(lambda x: orengo(x.split(' ')))

In [11]:
#total number of words
df1['words'] = df1['lyrics'].apply(lambda x: len(x.split(' ')))

In [12]:
#total number of words without considering stopwords
df1['words_ns'] = df1['no_stopwords'].apply(lambda x: len(x))

In [13]:
#removes stopwords from translated text and creates new column
df1['ns_pt'] = df1['translated'].apply(lambda x: remove_stopwords(x.split(' ')))

In [14]:
#vocabulary
df1['unique'] = df1['lyrics'].apply(lambda x: len(set(x.split(' '))))

In [15]:
#vocabulary after stemming
df1['orengo_unique'] = df1['orengo'].apply(lambda x: len(set(x)))

In [16]:
#vocabulary without considering stopwords
df1['ns_unique'] = df1['no_stopwords'].apply(lambda x: len(set(x)))

In [17]:
#vocabulary without considering stopwords after stemming
df1['ns_orengo_unique'] = df1['no_stopwords'].apply(lambda x: len(set(orengo(x))))

In [18]:
#vocabulary after translation
df1['unique_pt'] = df['translated'].apply(lambda x: len(set(x.split(' '))))

In [19]:
#vocabulary after stemming + translation
df1['orengo_pt'] = df['translated'].apply(lambda x: len(set(orengo(x))))

In [20]:
#vocabulary without considering stopwords (translated)
df1['ns_uni_pt'] = df1['ns_pt'].apply(lambda x: len(set(x)))

In [21]:
#vocabulary without considering stopwords after stemming + translation
df1['ns_or_pt'] = df1['ns_pt'].apply(lambda x: len(set(orengo(x))))

In [22]:
df1.head()

,artist,foreign,lyrics,translated,no_stopwords,orengo,words,words_ns,ns_pt,unique,orengo_unique,ns_unique,ns_orengo_unique,unique_pt,orengo_pt,ns_uni_pt,ns_or_pt
0,/aline-barros/,443,mestre eu preciso de um milagre transforma min...,mestre eu preciso de um milagre transforma min...,"[mestre, preciso, milagre, transforma, vida, e...","[mestr, eu, precis, de, um, milagr, transform,...",3826,2081,"[mestre, preciso, milagre, transforma, vida, e...",747,581,650,520,69,30,584,464
1,/amado-batista/,297,ao te ver pela primeira vez eu tremi todo uma ...,ao te ver pela primeira vez eu tremi todo uma ...,"[ver, primeira, vez, tremi, todo, coisa, tomou...","[ao, te, ver, pel, prim, vez, eu, trem, tod, u...",3305,1718,"[ver, primeira, vez, tremi, todo, coisa, tomou...",845,621,749,559,51,27,690,530
2,/ana-carolina/,535,eu e você não é assim tão complicado não é dif...,eu e você não é assim tão complicado não é dif...,"[assim, tão, complicado, difícil, perceber, do...","[eu, e, voc, não, é, assim, tão, complic, não,...",3907,2106,"[assim, tão, complicado, difícil, perceber, do...",775,599,685,539,58,30,630,497
3,/anavitoria/,681,eu poderia acordar sem teu olhar de sono sem t...,eu poderia acordar sem teu olhar de sono sem t...,"[poderia, acordar, olhar, sono, lábio, dono, q...","[eu, pod, acord, sem, teu, olh, de, son, sem, ...",4885,2614,"[poderia, acordar, olhar, sono, lábio, dono, q...",851,673,766,613,60,29,699,569
4,/anderson-freire/,505,não consigo ir além do teu olhar tudo que eu c...,não consigo ir além do teu olhar tudo que eu c...,"[consigo, ir, além, olhar, tudo, consigo, imag...","[não, consig, ir, além, do, teu, olh, tud, que...",5548,3028,"[consigo, ir, além, olhar, tudo, consigo, imag...",1126,868,1024,801,54,28,934,742


In [23]:
df1.to_csv('./data/final_data.csv', index=False)